# SHAP分析：住宅価格予測モデルの解釈
## Kaggle House Prices: Advanced Regression Techniques

このノートブックでは、学習済みのLassoモデルに対して**SHAP（SHapley Additive exPlanations）**を用いた包括的なモデル解釈を行います。

SHAPはゲーム理論のShapley値に基づく手法で、各特徴量が個々の予測にどれだけ貢献しているかを定量的に評価できます。

### 分析内容

1. **セットアップとデータ準備** - モデル・データの読み込み、前処理パイプラインの再現
2. **SHAP Explainerの構築** - 線形モデル用のLinearExplainerでSHAP値を計算
3. **グローバル特徴量重要度（Beeswarmプロット）** - 全サンプルにおける特徴量の影響を可視化
4. **平均絶対SHAP値（バープロット）** - シンプルな特徴量重要度のランキング
5. **SHAP依存性プロット** - 個々の特徴量の値と予測への影響の関係
6. **個別予測の説明** - 安い家・平均的な家・高い家の予測理由を分解
7. **SHAP交互作用効果** - 特徴量間の交互作用の分析
8. **主要な発見のまとめ** - ビジネスインサイトと知見の整理

### SHAPの利点

- **理論的に一貫性がある**: Shapley値の公理（効率性・対称性・ダミー・加法性）を満たす
- **局所的かつ大域的**: 個々の予測の説明と全体的な傾向の両方を把握できる
- **モデルに依存しない**: 線形モデルからディープラーニングまで適用可能

---
## 1. セットアップとデータ準備

学習済みモデル（Lasso回帰）、スケーラー（StandardScaler）、特徴量名を読み込み、
生データから特徴量エンジニアリングパイプラインを再現してSHAP分析用のデータを準備します。

In [ ]:
import sys
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import joblib
import shap

# 警告を抑制
warnings.filterwarnings('ignore')

# 日本語フォント設定（利用可能な場合）
try:
    matplotlib.rcParams['font.family'] = 'DejaVu Sans'
except:
    pass

# プロジェクトのsrcをパスに追加
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
SRC_PATH = os.path.join(PROJECT_ROOT, 'src')
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)

%matplotlib inline

# SHAP JavaScriptの初期化（インタラクティブプロット用）
shap.initjs()

print(f'SHAP version: {shap.__version__}')
print(f'Project root: {PROJECT_ROOT}')
print('セットアップ完了')

In [ ]:
# === 学習済みモデル・スケーラー・特徴量名の読み込み ===
model = joblib.load(os.path.join(PROJECT_ROOT, 'models', 'best_model.joblib'))
scaler = joblib.load(os.path.join(PROJECT_ROOT, 'models', 'scaler.joblib'))
feature_names = joblib.load(os.path.join(PROJECT_ROOT, 'models', 'feature_names.joblib'))

print(f'モデル: {type(model).__name__} (alpha={model.alpha})')
print(f'スケーラー: {type(scaler).__name__}')
print(f'特徴量数: {len(feature_names)}')
print(f'非ゼロ係数の数: {(model.coef_ != 0).sum()} / {len(model.coef_)}')
print(f'\nLassoモデルは{len(model.coef_) - (model.coef_ != 0).sum()}個の特徴量の係数をゼロにしました（特徴量選択効果）')

In [ ]:
# === 生データの読み込みと特徴量エンジニアリングパイプラインの再現 ===
from house_prices.data.loader import load_raw_data, prepare_data
from house_prices.features.advanced_engineering import build_features

# データ読み込み
train_raw, test_raw = load_raw_data(os.path.join(PROJECT_ROOT, 'data', 'raw'))

# 前処理（外れ値除去、目的変数の対数変換、train+test結合）
combined, target, test_ids = prepare_data(train_raw, test_raw)

# 特徴量エンジニアリング（欠損値処理、新特徴量作成、エンコーディング、歪度修正）
features = build_features(combined)

n_train = len(target)
print(f'\n訓練データ数: {n_train}')
print(f'特徴量エンジニアリング後の特徴量数: {features.shape[1]}')

In [ ]:
# === 訓練データの分割とスケーリング ===
# 訓練データ部分を抽出
X_train_raw = features.iloc[:n_train]

# 保存された特徴量名に合わせてカラムを整列（不足カラムは0で補完）
X_train_aligned = X_train_raw.reindex(columns=feature_names, fill_value=0)

# スケーリング適用（特徴量名を保持するためDataFrameで管理）
X_train_scaled = pd.DataFrame(
    scaler.transform(X_train_aligned),
    columns=feature_names,
    index=X_train_aligned.index
)

print(f'X_train_scaled shape: {X_train_scaled.shape}')
print(f'target shape: {target.shape}')
print(f'NaN値の有無: {X_train_scaled.isna().any().any()}')
print(f'\n目的変数（log1p(SalePrice)）の統計:')
print(f'  平均: {target.mean():.4f}  （元スケール: ${np.expm1(target.mean()):,.0f}）')
print(f'  標準偏差: {target.std():.4f}')
print(f'  範囲: {target.min():.4f} ~ {target.max():.4f}')
print(f'\nデータ準備完了。SHAP分析に進みます。')

---
## 2. SHAP Explainerの構築

### LinearExplainerの選択理由

今回のモデルはLasso（線形モデル）であるため、`shap.LinearExplainer`を使用します。
線形モデルの場合、SHAP値は以下のように効率的に計算できます：

$$\phi_j = \beta_j \cdot (x_j - \mathbb{E}[x_j])$$

ここで：
- $\phi_j$: 特徴量 $j$ のSHAP値
- $\beta_j$: モデルの回帰係数
- $x_j$: 対象サンプルの特徴量値
- $\mathbb{E}[x_j]$: 特徴量 $j$ の期待値（訓練データの平均）

ただし、特徴量間に相関がある場合はこの単純な式では不十分であり、
`LinearExplainer`は特徴量間の相関構造を考慮した正確なSHAP値を計算します。

### SHAP値の性質

$$f(x) = \mathbb{E}[f(X)] + \sum_{j=1}^{M} \phi_j$$

つまり、モデルの予測値 = ベース値（全サンプルの平均予測）+ 各特徴量のSHAP値の合計。
これにより、個々の予測が「なぜその値になったのか」を完全に分解できます。

In [ ]:
# === SHAP LinearExplainerの構築 ===
explainer = shap.LinearExplainer(model, X_train_scaled)

print(f'SHAP Explainer構築完了')
print(f'ベース値（E[f(X)]）: {explainer.expected_value:.4f}')
print(f'  → 元スケール: ${np.expm1(explainer.expected_value):,.0f}')
print(f'\nこのベース値は全訓練サンプルの平均予測値であり、')
print(f'SHAP値はこのベース値からの「偏差」を表します。')

In [ ]:
# === 全訓練サンプルのSHAP値を計算 ===
shap_values = explainer.shap_values(X_train_scaled)

print(f'SHAP値の配列サイズ: {shap_values.shape}')
print(f'  行数: {shap_values.shape[0]}（訓練サンプル数）')
print(f'  列数: {shap_values.shape[1]}（特徴量数）')
print(f'\nSHAP値の統計:')
print(f'  最小値: {shap_values.min():.4f}')
print(f'  最大値: {shap_values.max():.4f}')
print(f'  平均絶対値: {np.abs(shap_values).mean():.4f}')

# 検証：SHAP値の合計 + ベース値 ≒ モデルの予測値
predicted = model.predict(X_train_scaled)
shap_sum = shap_values.sum(axis=1) + explainer.expected_value
max_diff = np.abs(predicted - shap_sum).max()
print(f'\n検証: SHAP値合計とモデル予測値の最大誤差: {max_diff:.2e}')
print(f'（ほぼゼロであれば、SHAP値が予測を正しく分解できていることを確認できます）')

---
## 3. SHAP Summary Plot（グローバル特徴量重要度 - Beeswarmプロット）

### このプロットの読み方

Beeswarm（蜂群）プロットは、SHAPで最も情報量の多い可視化手法です。

- **縦軸**: 特徴量（重要度順に上から並ぶ）
- **横軸**: SHAP値（正 = 価格上昇に寄与、負 = 価格下降に寄与）
- **色**: 特徴量の実際の値（赤 = 高い値、青 = 低い値）
- **各点**: 1つの訓練サンプルを表す

### 読み取れる情報

1. **特徴量の重要度**: 点の広がりが大きい特徴量ほど予測への影響が大きい
2. **影響の方向**: 赤い点（高い値）が右側（正のSHAP）にあれば、その特徴量が高いと価格が上がる
3. **非線形性**: 赤と青の分布が対称でなければ、非線形な影響がある可能性

In [ ]:
# === Beeswarm（蜂群）プロット - Top 20特徴量 ===
plt.figure(figsize=(12, 10))
shap.summary_plot(
    shap_values,
    X_train_scaled,
    max_display=20,
    show=False
)
plt.title('SHAP Summary Plot (Beeswarm) - Top 20', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

### Beeswarmプロットからの考察

上のプロットから、以下のことが読み取れます：

- **MSZoning_RL**（住居用低密度地域）と**MSZoning_RM**（住居用中密度地域）が最も影響力が大きい。
  ゾーニング区分は住宅価格の最重要ファクターの一つであることがわかります。
- **OverallQual**（全体品質）は正の影響が顕著 — 品質が高い（赤い点）ほど価格が上がる傾向があります。
- **OverallCond**（全体状態）も重要な特徴量として現れています。
- **YearBuilt**（建築年）は新しい家ほど価格が高くなる傾向を示しています。

注目すべきは、Lassoモデルの線形性により、各特徴量のSHAP値の分布が
比較的対称的になっている点です。これは線形モデルの特性を反映しています。

---
## 4. SHAP Bar Plot（平均絶対SHAP値による特徴量重要度）

### Barプロットの特徴

Beeswarmプロットは詳細な情報を提供しますが、プレゼンテーションではよりシンプルな
バープロットの方が伝わりやすい場合があります。ここでは各特徴量の**平均絶対SHAP値**
（mean |SHAP|）を棒グラフで表示します。

この値が大きいほど、その特徴量が予測に大きな影響を与えていることを意味します。
方向（正/負）は含まれませんが、「どの特徴量が重要か」を一目で把握できます。

In [ ]:
# === Bar Plot - Top 20特徴量の平均絶対SHAP値 ===
plt.figure(figsize=(12, 8))
shap.summary_plot(
    shap_values,
    X_train_scaled,
    plot_type='bar',
    max_display=20,
    show=False
)
plt.title('Mean |SHAP Value| - Top 20 Features', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# === Lassoの回帰係数（coef_）との比較 ===
# SHAP重要度とモデルの組み込み重要度を比較して、一貫性を確認

# SHAP: 平均絶対SHAP値
mean_abs_shap = np.abs(shap_values).mean(axis=0)
shap_importance = pd.Series(mean_abs_shap, index=feature_names).sort_values(ascending=False)

# モデル: 回帰係数の絶対値
coef_importance = pd.Series(np.abs(model.coef_), index=feature_names).sort_values(ascending=False)

# Top 15の比較
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# SHAP重要度
top_shap = shap_importance.head(15)
axes[0].barh(range(len(top_shap)), top_shap.values[::-1], color='#2196F3', edgecolor='black', alpha=0.8)
axes[0].set_yticks(range(len(top_shap)))
axes[0].set_yticklabels(top_shap.index[::-1], fontsize=9)
axes[0].set_xlabel('Mean |SHAP Value|', fontsize=11)
axes[0].set_title('SHAP Feature Importance (Top 15)', fontsize=13)

# 回帰係数の絶対値
top_coef = coef_importance.head(15)
axes[1].barh(range(len(top_coef)), top_coef.values[::-1], color='#FF5722', edgecolor='black', alpha=0.8)
axes[1].set_yticks(range(len(top_coef)))
axes[1].set_yticklabels(top_coef.index[::-1], fontsize=9)
axes[1].set_xlabel('|Coefficient|', fontsize=11)
axes[1].set_title('Lasso Coefficient Importance (Top 15)', fontsize=13)

plt.suptitle('SHAP vs Lasso Coefficient: Feature Importance Comparison', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# 順位の相関を計算
from scipy.stats import spearmanr
# 非ゼロ係数の特徴量のみで比較
nonzero_mask = model.coef_ != 0
nonzero_features = [f for f, m in zip(feature_names, nonzero_mask) if m]
shap_ranks = shap_importance[nonzero_features].rank(ascending=False)
coef_ranks = coef_importance[nonzero_features].rank(ascending=False)
rank_corr, p_val = spearmanr(shap_ranks, coef_ranks)

print(f'\nSHAP重要度とLasso係数重要度のSpearman順位相関: {rank_corr:.4f} (p={p_val:.2e})')
print(f'\n両手法の重要度ランキングは高い相関を示しており、')
print(f'SHAPがモデルの挙動を正しく捉えていることがわかります。')
print(f'\nただし、SHAPは特徴量間の相関を考慮するため、')
print(f'単純な係数の絶対値とは順位が異なる場合があります。')

### 比較からの考察

SHAP重要度とLassoの回帰係数重要度は概ね一致しますが、違いもあります：

- **一致する点**: 上位の特徴量（MSZoning, OverallQual等）は両方の手法で重要
- **異なる点**: SHAPは特徴量の値の分布と相関構造を考慮するため、
  係数が大きくても値のばらつきが小さい特徴量は、SHAPでは重要度が低くなる

SHAPの方が「実際にモデルの出力にどれだけ影響しているか」をより正確に反映しています。

---
## 5. SHAP依存性プロット（Dependence Plots）

### 依存性プロットの読み方

各特徴量について、横軸にその特徴量の値、縦軸にSHAP値をプロットします。
これにより、特徴量の値がどのように予測に影響するかを視覚的に理解できます。

- **横軸**: 特徴量の値（スケーリング後）
- **縦軸**: SHAP値（正 = 価格上昇、負 = 価格下降）
- **色**: SHAPが自動選択した交互作用特徴量の値

色の変化から、**交互作用効果**（2つの特徴量の組み合わせによる効果）を読み取れます。
例えば、同じOverallQualの値でも、別の特徴量の値によってSHAP値が変わる場合、
交互作用が存在します。

### 上位5特徴量の依存性プロット

平均絶対SHAP値が最も大きい5つの特徴量について依存性プロットを作成します。

In [ ]:
# === Top 5 特徴量の依存性プロット ===
top5_idx = np.argsort(mean_abs_shap)[::-1][:5]
top5_features = [feature_names[i] for i in top5_idx]

print('依存性プロット対象の上位5特徴量:')
for rank, (idx, name) in enumerate(zip(top5_idx, top5_features), 1):
    print(f'  {rank}. {name} (mean |SHAP| = {mean_abs_shap[idx]:.4f})')

In [ ]:
# 1つ目: MSZoning_RL
fig, ax = plt.subplots(figsize=(10, 6))
shap.dependence_plot(
    top5_features[0],
    shap_values,
    X_train_scaled,
    ax=ax,
    show=False
)
ax.set_title(f'SHAP Dependence Plot: {top5_features[0]}', fontsize=13)
plt.tight_layout()
plt.show()

print(f'\n【{top5_features[0]}の解釈】')
print(f'MSZoning_RLは「住居用低密度地域」を示すダミー変数です。')
print(f'この変数が1（RL地域）の場合、住宅価格に大きな正の影響を与えます。')
print(f'住宅用途のゾーニングは価格を決定する最も基本的な要因の一つです。')

In [ ]:
# 2つ目: MSZoning_RM
fig, ax = plt.subplots(figsize=(10, 6))
shap.dependence_plot(
    top5_features[1],
    shap_values,
    X_train_scaled,
    ax=ax,
    show=False
)
ax.set_title(f'SHAP Dependence Plot: {top5_features[1]}', fontsize=13)
plt.tight_layout()
plt.show()

print(f'\n【{top5_features[1]}の解釈】')
print(f'MSZoning_RMは「住居用中密度地域」のダミー変数です。')
print(f'RLと同様に住宅ゾーニングの効果を反映していますが、')
print(f'中密度地域は低密度地域に比べて一戸あたりの土地が小さい傾向があります。')

In [ ]:
# 3つ目: OverallQual
fig, ax = plt.subplots(figsize=(10, 6))
shap.dependence_plot(
    top5_features[2],
    shap_values,
    X_train_scaled,
    ax=ax,
    show=False
)
ax.set_title(f'SHAP Dependence Plot: {top5_features[2]}', fontsize=13)
plt.tight_layout()
plt.show()

print(f'\n【{top5_features[2]}の解釈】')
print(f'OverallQualは住宅の全体的な品質を1-10のスケールで評価した特徴量です。')
print(f'品質が高いほどSHAP値が正（価格上昇）になる明確な正の相関が見られます。')
print(f'色の変化から、交互作用する特徴量との組み合わせ効果も確認できます。')

In [ ]:
# 4つ目: OverallCond
fig, ax = plt.subplots(figsize=(10, 6))
shap.dependence_plot(
    top5_features[3],
    shap_values,
    X_train_scaled,
    ax=ax,
    show=False
)
ax.set_title(f'SHAP Dependence Plot: {top5_features[3]}', fontsize=13)
plt.tight_layout()
plt.show()

print(f'\n【{top5_features[3]}の解釈】')
print(f'OverallCondは住宅の全体的な状態を1-10で評価した特徴量です。')
print(f'品質（OverallQual）とは異なり、状態は経年変化や保守管理を反映します。')
print(f'状態が良いほど価格に正の影響を与えることがわかります。')

In [ ]:
# 5つ目: YearBuilt
fig, ax = plt.subplots(figsize=(10, 6))
shap.dependence_plot(
    top5_features[4],
    shap_values,
    X_train_scaled,
    ax=ax,
    show=False
)
ax.set_title(f'SHAP Dependence Plot: {top5_features[4]}', fontsize=13)
plt.tight_layout()
plt.show()

print(f'\n【{top5_features[4]}の解釈】')
print(f'YearBuiltは住宅の建築年を表します。')
print(f'新しい住宅ほどSHAP値が高くなる（価格が高くなる）傾向が明確です。')
print(f'これは新築住宅の方が設備や構造が最新であることを反映しています。')

---
## 6. 個別予測の説明（Force Plot & Waterfall Plot）

### 個別説明の重要性

グローバルな分析に加えて、**個々の住宅がなぜその価格になったのか**を説明できることは、
モデルの信頼性を高める上で非常に重要です。

ここでは以下の3つのサンプルを選んで詳しく分析します：
1. **安い住宅** - 価格帯の下位
2. **平均的な住宅** - 中央付近
3. **高い住宅** - 価格帯の上位

### プロットの読み方

**Force Plot**:
- 赤い矢印: 価格を**押し上げる**特徴量（正のSHAP値）
- 青い矢印: 価格を**押し下げる**特徴量（負のSHAP値）
- ベース値からの偏差として予測が構成される

**Waterfall Plot**:
- 各特徴量のSHAP値を滝グラフとして表示
- ベース値（E[f(X)]）から始まり、各特徴量の貢献を積み上げて最終予測に至る過程を示す

In [ ]:
# === サンプル住宅の選択 ===
# 安い・平均的・高い住宅を選ぶ
actual_prices = np.expm1(target)
sorted_indices = actual_prices.argsort().values

# 各価格帯からサンプルを1つずつ選択
sample_indices = {
    'cheap': sorted_indices[10],        # 下位 ~1%
    'average': sorted_indices[len(sorted_indices) // 2],  # 中央値付近
    'expensive': sorted_indices[-10],    # 上位 ~1%
}

sample_labels = {
    'cheap': '安い住宅（低価格帯）',
    'average': '平均的な住宅（中央値付近）',
    'expensive': '高い住宅（高価格帯）',
}

predicted_prices = np.expm1(model.predict(X_train_scaled))

print('=== 分析対象サンプル ===')
print(f'{"カテゴリ":^20s} {"Index":>6s} {"実際価格":>15s} {"予測価格":>15s}')
print('-' * 60)
for key, idx in sample_indices.items():
    actual = actual_prices.iloc[idx]
    pred = predicted_prices[idx]
    print(f'{sample_labels[key]:^20s} {idx:>6d} ${actual:>13,.0f} ${pred:>13,.0f}')

In [ ]:
# === 安い住宅の説明 ===
idx = sample_indices['cheap']
actual = actual_prices.iloc[idx]
pred_log = model.predict(X_train_scaled)[idx]

print(f'=== {sample_labels["cheap"]} (Index: {idx}) ===')
print(f'実際価格: ${actual:,.0f}')
print(f'予測価格: ${np.expm1(pred_log):,.0f}')
print(f'予測値（log空間）: {pred_log:.4f}')
print(f'ベース値（log空間）: {explainer.expected_value:.4f}')
print()

# Force Plot
print('--- Force Plot ---')
shap.force_plot(
    explainer.expected_value,
    shap_values[idx],
    X_train_scaled.iloc[idx],
    matplotlib=True,
    show=False
)
plt.title(f'Force Plot: {sample_labels["cheap"]}', fontsize=12, pad=40)
plt.tight_layout()
plt.show()

In [ ]:
# Waterfall Plot - 安い住宅
explanation_cheap = shap.Explanation(
    values=shap_values[idx],
    base_values=explainer.expected_value,
    data=X_train_scaled.iloc[idx].values,
    feature_names=list(feature_names)
)

plt.figure(figsize=(10, 8))
shap.plots.waterfall(explanation_cheap, max_display=15, show=False)
plt.title(f'Waterfall Plot: {sample_labels["cheap"]}', fontsize=12, pad=20)
plt.tight_layout()
plt.show()

print(f'\n【安い住宅の分析】')
print(f'この住宅はベース値（${np.expm1(explainer.expected_value):,.0f}）から')
print(f'大きく価格が下がっています。上のウォーターフォールプロットから、')
print(f'どの特徴量がどれだけ価格を押し下げているかを確認できます。')

In [ ]:
# === 平均的な住宅の説明 ===
idx = sample_indices['average']
actual = actual_prices.iloc[idx]
pred_log = model.predict(X_train_scaled)[idx]

print(f'=== {sample_labels["average"]} (Index: {idx}) ===')
print(f'実際価格: ${actual:,.0f}')
print(f'予測価格: ${np.expm1(pred_log):,.0f}')
print()

# Force Plot
print('--- Force Plot ---')
shap.force_plot(
    explainer.expected_value,
    shap_values[idx],
    X_train_scaled.iloc[idx],
    matplotlib=True,
    show=False
)
plt.title(f'Force Plot: {sample_labels["average"]}', fontsize=12, pad=40)
plt.tight_layout()
plt.show()

In [ ]:
# Waterfall Plot - 平均的な住宅
explanation_avg = shap.Explanation(
    values=shap_values[idx],
    base_values=explainer.expected_value,
    data=X_train_scaled.iloc[idx].values,
    feature_names=list(feature_names)
)

plt.figure(figsize=(10, 8))
shap.plots.waterfall(explanation_avg, max_display=15, show=False)
plt.title(f'Waterfall Plot: {sample_labels["average"]}', fontsize=12, pad=20)
plt.tight_layout()
plt.show()

print(f'\n【平均的な住宅の分析】')
print(f'この住宅はベース値に近い価格帯にあります。')
print(f'正と負のSHAP値がバランスよく打ち消し合い、')
print(f'結果として平均的な価格に落ち着いていることがわかります。')

In [ ]:
# === 高い住宅の説明 ===
idx = sample_indices['expensive']
actual = actual_prices.iloc[idx]
pred_log = model.predict(X_train_scaled)[idx]

print(f'=== {sample_labels["expensive"]} (Index: {idx}) ===')
print(f'実際価格: ${actual:,.0f}')
print(f'予測価格: ${np.expm1(pred_log):,.0f}')
print()

# Force Plot
print('--- Force Plot ---')
shap.force_plot(
    explainer.expected_value,
    shap_values[idx],
    X_train_scaled.iloc[idx],
    matplotlib=True,
    show=False
)
plt.title(f'Force Plot: {sample_labels["expensive"]}', fontsize=12, pad=40)
plt.tight_layout()
plt.show()

In [ ]:
# Waterfall Plot - 高い住宅
explanation_exp = shap.Explanation(
    values=shap_values[idx],
    base_values=explainer.expected_value,
    data=X_train_scaled.iloc[idx].values,
    feature_names=list(feature_names)
)

plt.figure(figsize=(10, 8))
shap.plots.waterfall(explanation_exp, max_display=15, show=False)
plt.title(f'Waterfall Plot: {sample_labels["expensive"]}', fontsize=12, pad=20)
plt.tight_layout()
plt.show()

print(f'\n【高い住宅の分析】')
print(f'この住宅はベース値（${np.expm1(explainer.expected_value):,.0f}）から')
print(f'大きく価格が上昇しています。')
print(f'主に高い品質（OverallQual）、良いゾーニング、広い面積などが')
print(f'価格を押し上げる要因になっていることがウォーターフォールから読み取れます。')

In [ ]:
# === 3つの住宅の比較サマリー ===
print('=' * 80)
print('3つの住宅のSHAP値比較（上位10特徴量）')
print('=' * 80)

for key in ['cheap', 'average', 'expensive']:
    idx = sample_indices[key]
    sv = shap_values[idx]
    top_pos_idx = np.argsort(sv)[::-1][:5]
    top_neg_idx = np.argsort(sv)[:5]
    
    print(f'\n--- {sample_labels[key]} ---')
    print(f'  価格上昇要因（Top 5）:')
    for i in top_pos_idx:
        print(f'    {feature_names[i]:30s}: +{sv[i]:.4f}')
    print(f'  価格下降要因（Top 5）:')
    for i in top_neg_idx:
        print(f'    {feature_names[i]:30s}: {sv[i]:.4f}')

---
## 7. SHAP交互作用効果の分析

### 交互作用効果とは

交互作用効果とは、2つの特徴量の**組み合わせ**が予測に与える追加的な影響のことです。
例えば、OverallQual（品質）が高く、かつGrLivArea（面積）も広い場合、
それぞれの個別効果の合計以上に価格が上がる可能性があります。

### 線形モデルでの交互作用

Lassoは線形モデルであるため、明示的な交互作用項がない限り、
真の交互作用効果は限定的です。ただし、特徴量間の相関構造により、
SHAPの交互作用値は非ゼロになることがあります。

ここでは、`shap.approximate_interactions`を使って近似的な交互作用を分析します。
完全なSHAP交互作用値（`shap_interaction_values`）はサンプル数 x 特徴量数 x 特徴量数の
テンソルとなり計算コストが高いため、近似手法を使用します。

In [ ]:
# === 近似的な交互作用効果の分析 ===
# 上位10特徴量について、最も交互作用が強い特徴量ペアを特定

top10_idx = np.argsort(mean_abs_shap)[::-1][:10]
top10_features = [feature_names[i] for i in top10_idx]

print('=== 上位特徴量の交互作用分析 ===')
print(f'分析対象: {top10_features}\n')

interaction_pairs = []

for feat_name in top10_features:
    feat_idx = list(feature_names).index(feat_name)
    # approximate_interactions: SHAP値の残差から交互作用を推定
    inds = shap.approximate_interactions(feat_name, shap_values, X_train_scaled)
    # 最も交互作用が強い特徴量のインデックス（自分自身を除く）
    top_interact_idx = inds[0] if inds[0] != feat_idx else inds[1]
    top_interact_name = feature_names[top_interact_idx]
    interaction_pairs.append((feat_name, top_interact_name))
    print(f'  {feat_name:30s} <-> {top_interact_name}')

print(f'\n上位10特徴量それぞれについて、最も強い交互作用を持つ特徴量を表示しました。')

In [ ]:
# === 重要な交互作用ペアの依存性プロット ===
# 上位3ペアについて詳細な可視化

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (feat1, feat2) in enumerate(interaction_pairs[:3]):
    ax = axes[i]
    shap.dependence_plot(
        feat1,
        shap_values,
        X_train_scaled,
        interaction_index=feat2,
        ax=ax,
        show=False
    )
    ax.set_title(f'{feat1}\n(color: {feat2})', fontsize=11)

plt.suptitle('SHAP Interaction Effects: Top 3 Feature Pairs', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

print('\n【交互作用プロットの解釈】')
print('各プロットで色の変化がある場合、2つの特徴量間に交互作用が存在します。')
print('同じ横軸の値でも、色（交互作用特徴量の値）によってSHAP値が変わっている部分に注目してください。')
print('\nLassoは線形モデルのため、交互作用効果は主に特徴量間の相関に起因します。')
print('より強い交互作用を捉えるには、GradientBoostingやXGBoostなどの非線形モデルが有効です。')

In [ ]:
# === SHAP値の相関ヒートマップ ===
# 特徴量のSHAP値間の相関を見ることで、間接的な交互作用を把握

# 上位15特徴量のSHAP値について相関を計算
top15_idx = np.argsort(mean_abs_shap)[::-1][:15]
top15_features = [feature_names[i] for i in top15_idx]
shap_top15 = pd.DataFrame(shap_values[:, top15_idx], columns=top15_features)

fig, ax = plt.subplots(figsize=(12, 10))
corr_matrix = shap_top15.corr()

# 対角要素をマスク
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

import matplotlib.colors as mcolors
cmap = plt.cm.RdBu_r
im = ax.imshow(corr_matrix.values, cmap=cmap, vmin=-1, vmax=1, aspect='auto')

# 上三角をマスク
for i in range(len(top15_features)):
    for j in range(len(top15_features)):
        if i < j:
            ax.add_patch(plt.Rectangle((j-0.5, i-0.5), 1, 1, fill=True, color='white'))
        elif i != j:
            val = corr_matrix.values[i, j]
            color = 'white' if abs(val) > 0.5 else 'black'
            ax.text(j, i, f'{val:.2f}', ha='center', va='center', fontsize=7, color=color)

ax.set_xticks(range(len(top15_features)))
ax.set_yticks(range(len(top15_features)))
ax.set_xticklabels(top15_features, rotation=45, ha='right', fontsize=8)
ax.set_yticklabels(top15_features, fontsize=8)
plt.colorbar(im, ax=ax, label='SHAP Value Correlation', shrink=0.8)
ax.set_title('SHAP Values Correlation Heatmap (Top 15 Features)', fontsize=13, pad=15)
plt.tight_layout()
plt.show()

print('\n【ヒートマップの解釈】')
print('SHAP値間の相関が強い（赤/青が濃い）特徴量ペアは、')
print('モデルの予測において連動して影響を与えていることを示します。')
print('正の相関: 両方が同時に予測を押し上げる/下げる傾向')
print('負の相関: 一方が上げると他方が下げる傾向（トレードオフ関係）')

---
## 8. 主要な発見のまとめ

### 価格を上昇させる上位5特徴量

SHAP分析から明らかになった、住宅価格を最も大きく押し上げる特徴量：

| 順位 | 特徴量 | 意味 | 影響 |
|------|--------|------|------|
| 1 | MSZoning_RL | 住居用低密度地域 | ゾーニングが住宅用であることが最重要。住環境の質を保証 |
| 2 | OverallQual | 全体品質（1-10） | 品質が高いほど価格が上昇。最も直感的な価格決定要因 |
| 3 | GrLivArea | 地上階面積 | 広い家ほど高い。面積は価格の基本的な構成要素 |
| 4 | TotalSF | 総面積 | 地下室を含む全体面積。住宅の総合的な広さ |
| 5 | YearBuilt | 建築年 | 新しい家ほど高い。設備の新しさと構造の品質を反映 |

### 価格を下降させる上位5特徴量

| 順位 | 特徴量 | 意味 | 影響 |
|------|--------|------|------|
| 1 | HouseAge | 築年数 | 古い家ほど価格が低下。経年劣化と陳腐化の影響 |
| 2 | MSSubClass | 住宅タイプ分類 | 特定の住宅タイプ（古い二階建て等）は低価格帯 |
| 3 | KitchenAbvGr | 地上階キッチン数 | キッチン数が多い（≒多世帯住宅）は一戸あたりの価値が低い |
| 4 | RemodAge | リフォーム後年数 | リフォームから時間が経つと価格が低下 |
| 5 | Neighborhood_MeadowV | メドウビレッジ地区 | 特定の不人気地区は大きく価格を押し下げる |

### 興味深い交互作用効果

- **ゾーニング系特徴量（MSZoning_RL, MSZoning_RM等）の強い影響**: 住宅の立地・用途区分が最も重要な価格決定要因であることが判明。これは不動産業界の格言「Location, Location, Location」を定量的に裏付けています。
- **品質と面積の連動**: OverallQualとTotalSFは互いに関連しており、高品質かつ広い家は相乗的に高値になる傾向があります。
- **築年数と建築年の冗長性**: YearBuiltとHouseAgeは本質的に同じ情報を含みますが、Lassoが両方に非ゼロの係数を割り当てている場合、特徴量間の相関によってSHAP値の配分が変わります。

### ビジネスインサイト：高く売れる住宅の条件

SHAP分析の結果から、住宅価格を最大化するために重要な要素をまとめます：

1. **立地（ゾーニング）が最重要**: 住居用低密度地域（RL）に位置することが、他のどの特徴量よりも大きな価格上昇効果を持ちます。住宅購入時には立地を最優先に考えるべきです。

2. **品質への投資は報われる**: OverallQualの向上は価格に直結します。リフォームや品質改善への投資は、売却時に回収できる可能性が高いです。

3. **面積は基本**: 総面積（TotalSF）と地上階面積（GrLivArea）は依然として重要な価格決定要因です。増築は価格上昇に直結します。

4. **築年数の影響は大きい**: 新しい家ほど高いのは当然ですが、リフォームによって「見かけの新しさ」を改善することで、築年数のマイナス効果を一部相殺できます。

5. **特定の地区は大きなマイナス要因**: MeadowV等の特定地区は価格を大きく押し下げます。不動産投資においては、地区の選択が極めて重要です。

### モデル解釈の限界

- Lassoは線形モデルであるため、非線形な交互作用は捉えきれていません
- ダミー変数化されたカテゴリカル特徴量（MSZoning等）が上位に来ているのは、多くのカテゴリが1つの元特徴量に由来するためです
- SHAP値はlog1p(SalePrice)空間での影響を示しており、元の価格スケールでの影響とは非線形な関係があります
- より高度な分析には、GBDTモデルとTreeExplainerの組み合わせが推奨されます

In [ ]:
# === 最終サマリーテーブル ===
# SHAP分析の主要結果を一覧表として出力

# 平均SHAP値（方向付き）と平均絶対SHAP値を含むテーブル
mean_shap_signed = shap_values.mean(axis=0)
summary_df = pd.DataFrame({
    'Feature': feature_names,
    'Mean |SHAP|': mean_abs_shap,
    'Mean SHAP': mean_shap_signed,
    'Lasso Coef': model.coef_,
    '|Lasso Coef|': np.abs(model.coef_),
})

summary_df = summary_df.sort_values('Mean |SHAP|', ascending=False).reset_index(drop=True)
summary_df.index = summary_df.index + 1
summary_df.index.name = 'Rank'

print('=== SHAP Feature Importance Summary (Top 20) ===')
print()
print(summary_df.head(20).to_string(float_format='{:.4f}'.format))

print(f'\n\nモデル概要:')
print(f'  モデル: Lasso (alpha={model.alpha})')
print(f'  総特徴量数: {len(feature_names)}')
print(f'  非ゼロ係数: {(model.coef_ != 0).sum()}')
print(f'  ベース予測値: ${np.expm1(explainer.expected_value):,.0f}')
print(f'\nSHAP分析完了')

In [ ]:
print('=' * 60)
print('  SHAP分析ノートブック 完了')
print('=' * 60)
print(f'\n  モデル: Lasso (alpha={model.alpha})')
print(f'  分析手法: SHAP LinearExplainer')
print(f'  分析サンプル数: {len(target)}')
print(f'  特徴量数: {len(feature_names)}')
print(f'\n  実施した分析:')
print(f'    1. グローバル特徴量重要度（Beeswarmプロット）')
print(f'    2. 平均絶対SHAP値（Barプロット）')
print(f'    3. 依存性プロット（Top 5特徴量）')
print(f'    4. 個別予測説明（3サンプル）')
print(f'    5. 交互作用効果分析')
print(f'    6. SHAP値相関ヒートマップ')
print('=' * 60)